## Using Python to Query MySQL
This notebook demonstrates using a couple of different database connectivity libraries to connect to and query a MySQL database.
- **PyMySQL** library
- MySQL's Native **mysql.connector** library
- **SqlAlchemy** library

### 1.0. Prerequisites

#### 1.1. First, you must **install** the libaries into your *python* environment by executing the following commands in a *Terminal window*
- \$ python3 -m pip install PyMySQL
- \$ python3 -m pip install mysql-connector-python
- \$ python3 -m pip install sqlalchemy

#### 1.2. Next, as with all Jupyter Notebooks, you need to **Import** the libaries that you'll be working with in the notebook,

In [1]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### 1.3. And then, Assign Connection Variables for the MySQL Server & Database with which You'll be Working 

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root"
pwd = "Passw0rd123"
db_name = "northwind"

### 2.0. Using the PyMySQL Library
#### 2.1. Using a Cursor to Iterate the Rows Returned

In [3]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM products;')
    
    for row in cursor.fetchmany(size=5):
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

('4', 1, 'NWTB-1', 'Northwind Traders Chai', None, Decimal('13.5000'), Decimal('18.0000'), 10, 40, '10 boxes x 20 bags', 0, 10, 'Beverages', b'')
('10', 3, 'NWTCO-3', 'Northwind Traders Syrup', None, Decimal('7.5000'), Decimal('10.0000'), 25, 100, '12 - 550 ml bottles', 0, 25, 'Condiments', b'')
('10', 4, 'NWTCO-4', 'Northwind Traders Cajun Seasoning', None, Decimal('16.5000'), Decimal('22.0000'), 10, 40, '48 - 6 oz jars', 0, 10, 'Condiments', b'')
('10', 5, 'NWTO-5', 'Northwind Traders Olive Oil', None, Decimal('16.0125'), Decimal('21.3500'), 10, 40, '36 boxes', 0, 10, 'Oil', b'')
('2;6', 6, 'NWTJP-6', 'Northwind Traders Boysenberry Spread', None, Decimal('18.7500'), Decimal('25.0000'), 25, 100, '12 - 8 oz jars', 0, 25, 'Jams, Preserves', b'')


In [4]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor(pymysql.cursors.DictCursor)

try:
    cursor.execute('SELECT * FROM products ORDER BY list_price DESC LIMIT 5;')
    
    for row in cursor.fetchall():
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

{'supplier_ids': '2;6', 'id': 20, 'product_code': 'NWTJP-6', 'product_name': 'Northwind Traders Marmalade', 'description': None, 'standard_cost': Decimal('60.7500'), 'list_price': Decimal('81.0000'), 'reorder_level': 10, 'target_level': 40, 'quantity_per_unit': '30 gift boxes', 'discontinued': 0, 'minimum_reorder_quantity': 10, 'category': 'Jams, Preserves', 'attachments': b''}
{'supplier_ids': '2', 'id': 51, 'product_code': 'NWTDFN-51', 'product_name': 'Northwind Traders Dried Apples', 'description': None, 'standard_cost': Decimal('39.7500'), 'list_price': Decimal('53.0000'), 'reorder_level': 10, 'target_level': 40, 'quantity_per_unit': '50 - 300 g pkgs.', 'discontinued': 0, 'minimum_reorder_quantity': 10, 'category': 'Dried Fruit & Nuts', 'attachments': b''}
{'supplier_ids': '3;4', 'id': 43, 'product_code': 'NWTB-43', 'product_name': 'Northwind Traders Coffee', 'description': None, 'standard_cost': Decimal('34.5000'), 'list_price': Decimal('46.0000'), 'reorder_level': 25, 'target_lev

#### 2.2. Using the Pandas read_sql() Method to Return a DataFrame

In [5]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)

df = pd.read_sql("SELECT * FROM products ORDER BY list_price DESC;", conn)

conn.close()
df.head()

,supplier_ids,id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments
0,2;6,20,NWTJP-6,Northwind Traders Marmalade,None,60.75,81.0,10,40,30 gift boxes,0,10.0,"Jams, Preserves",b''
1,2,51,NWTDFN-51,Northwind Traders Dried Apples,None,39.75,53.0,10,40,50 - 300 g pkgs.,0,10.0,Dried Fruit & Nuts,b''
2,3;4,43,NWTB-43,Northwind Traders Coffee,None,34.50,46.0,25,100,16 - 500 g tins,0,25.0,Beverages,b''
3,8,8,NWTS-8,Northwind Traders Curry Sauce,None,30.00,40.0,10,40,12 - 12 oz jars,0,10.0,Sauces,b''
4,6,17,NWTCFV-17,Northwind Traders Fruit Cocktail,None,29.25,39.0,10,40,15.25 OZ,0,10.0,Canned Fruit & Vegetables,b''


In [6]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN %s AND %s
    ORDER BY list_price DESC;
"""

In [7]:
conn = mysql.connector.connect(user=user_id, password=pwd,
                               host=host_ip, database=db_name,
                               auth_plugin='mysql_native_password') #caching_sha2_password mysql_native_password

AttributeError: module 'mysql.connector' has no attribute 'connect'

In [ ]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    for (product_id, product_name, list_price) in cursor:
        print("{}, {} was sold for {}".format(product_id, product_name, list_price))
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")   

In [ ]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    row = cursor.fetchone() 
    while row:
        print(row)
        row = cursor.fetchone()
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")

In [ ]:
conn.close()

#### 3.2. Using Pandas read_sql() Method to Return a DataFrame

In [ ]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN {0} AND {1}
    ORDER BY list_price DESC;
""".format(lbound, ubound)

print(sql_query)

In [ ]:
configs = {
    'user': user_id,
    'password': pwd,
    'host': host_name,
    'database': db_name,
    'auth_plugin': 'mysql_native_password',
    'raise_on_warnings': True
}

conn = mysql.connector.connect(**configs)

df = pd.read_sql(sql_query, conn)

conn.close()
df.tail()

### 5.0 Define Helper Functions to Encapsulate and Abstract the Implementation Details

In [ ]:
sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    ORDER BY list_price DESC;
"""

#### 5.1. Using Individual Connection Parameters

In [8]:
def get_pymysql_dataframe(host, user, password, database_name, sql_query_string):
    connection = pymysql.connect(host=host, user=user, password=password, database=database_name)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [9]:
df = get_pymysql_dataframe(host_name, user_id, pwd, db_name, sql_query)
df.head()

DatabaseError: Execution failed on sql '
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN %s AND %s
    ORDER BY list_price DESC;
': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '%s AND %s\n    ORDER BY list_price DESC' at line 5")

In [ ]:
print("Shape: {}\n".format(df.shape))

#### 5.1.1. Using SqlAlchemy

In [ ]:
def get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [ ]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query)
df.head()

In [ ]:
print(f"Shape: {df.shape[0]} Observations x {df.shape[1]} Features")

In [ ]:
print(f"Shape: {dframe.shape[0]} Observations x {dframe.shape[1]} Features")

### 6.0. Writing a Pandas DataFrame to a SQL Database

In [ ]:
def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()

In [ ]:
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, dframe, 'dim_products')

In [ ]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, 'SELECT * FROM dim_products')
df.head()

### 7.0. Explore Pandas DataFrames' Capabilities
#### 7.1. Display the Data Type of Each Feature

In [ ]:
sql_query = "SELECT * FROM northwind.products;"

df = get_mysql_dataframe(sql_query, configs)

In [ ]:
df.dtypes

#### 7.1. Inspect the Cardinality (number of unique values) of each Feature

In [ ]:
df.nunique()

In [ ]:
unique_values = []

for column in df.columns:
    unique_values.append(df[column].unique())
    
data = list(zip(df.columns, unique_values))    
    
pd.DataFrame(data, columns=['Feature', 'Unique Values'])

#### 7.2. Display any Missing (NULL) values

In [ ]:
df.isnull().sum().sort_values(ascending=True)

#### 7.3. Separate Numerical and Categorical Features

In [ ]:
numerical_cols = [col for col in df.columns if df.dtypes[col] != 'O']
categorical_cols = [col for col in df.columns if col not in numerical_cols]

print(numerical_cols)
print(categorical_cols)

#### 7.4. Evaluate the Statistical Distribution of the Numerical Features

In [ ]:
df[numerical_cols].describe()

#### 7.5. Write the Contents of the DataFrame to a Comma-Separated Values (CSV) File

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data')
dest_file = os.path.join(data_dir, 'northwind_products.csv')

df.to_csv(dest_file)